In [ ]:
#Install spacy and geopy
!python -m spacy download en_core_web_sm
!pip install geopy folium




  Using cached https://github.com/explosion/spacy-models/releases/download/en_core_web_sm-3.8.0/en_core_web_sm-3.8.0-py3-none-any.whl (12.8 MB)
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [12]:
#Import request that you will use to retrieve the odyssey texts. We will then take the url for the text and use request to open the text.

import requests

url = "https://www.gutenberg.org/files/1727/1727-0.txt"
odyssey_text = requests.get(url).text


In [ ]:
#
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp(odyssey_text)

locations = [ent.text for ent in doc.ents if ent.label_ == "GPE"]
unique_locations = list(set(locations))


In [ ]:
'''
'''
from geopy.geocoders import Nominatim
from time import sleep

geolocator = Nominatim(user_agent="odyssey_mapper")
location_data = []

for loc in unique_locations:
    try:
        geo = geolocator.geocode(loc)
        if geo:
            location_data.append({
                "name": loc,
                "latitude": geo.latitude,
                "longitude": geo.longitude
            })
            sleep(1)
    except Exception as e:
        print(f"Failed on {loc}: {e}")


Failed on a United States: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=a+United+States&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))
Failed on Hermes: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /search?q=Hermes&format=json&limit=1 (Caused by ReadTimeoutError("HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Read timed out. (read timeout=1)"))


In [15]:
import folium

map = folium.Map(location=[37.9838, 23.7275], zoom_start=5)  # center on Greece

for place in location_data:
    folium.Marker(
        [place["latitude"], place["longitude"]],
        popup=place["name"]
    ).add_to(map)

map.save("odyssey_map.html")
